# IndividualStopLoss logic

In [41]:
from utils.utils import get_open_positions, get_account

# summed is a dictionary of epics aggregated on their profit loss {'epic_name': profit loss value}
summed = {}
translations = {}

# loop through all open positions and and aggregate on 'epic' and SUM of profit loss
for i in get_open_positions():

    # find direction and price
    direction = i['position']['direction']

    # find profit loss
    if direction == 'BUY':
        price = i['market']['bid']
        pl = price - i['position']['openLevel']
    elif direction == 'SELL':
        price = i['market']['offer']
        pl = i['position']['openLevel'] - price

    # find epic and instrument name
    epic = i['market']['epic']
    instrument_name = i['market']['instrumentName']

    # add epic and instrument name to the translations dictionary (if it isn't in already)
    if not translations.get(epic):
        translations[epic] = instrument_name
    else:
        pass

    
    # if epic isn't in the summed dictionary yet, add it in with the pl
    if not summed.get(epic):
        summed[epic] = {'pl': pl, 'positions': [i]}
    # if epic is already in the summed dictionary, add profit loss to existing profit loss for epic
    else:
        summed[epic] = {'pl': summed[epic]['pl'] + pl, 'positions': summed[epic]['positions'] + [i]}

print(summed)
balance = get_account()['balance']

# go through all the epics in summed and close if pl <-0.01
for epic, pl in summed.items():
    pl_ratio = pl['pl']/balance
    if pl_ratio < -0.01:
        # TODO add in closing function with email
        print('close', epic, pl_ratio, pl['pl'], balance)
print(translations)

{'IX.D.NASDAQ.CASH.IP': {'pl': -1148.300000000001, 'positions': [{'position': {'contractSize': 1.0, 'createdDate': '2022/05/26 20:18:41:000', 'dealId': 'DIAAAAJHJDUHYBC', 'dealSize': 2.0, 'direction': 'SELL', 'limitLevel': None, 'openLevel': 12244.8, 'currency': 'GBP', 'controlledRisk': False, 'stopLevel': None, 'trailingStep': None, 'trailingStopDistance': None, 'limitedRiskPremium': None}, 'market': {'instrumentName': 'US Tech 100', 'expiry': 'DFB', 'epic': 'IX.D.NASDAQ.CASH.IP', 'instrumentType': 'INDICES', 'lotSize': 1.0, 'high': 12881.9, 'low': 12670.0, 'percentageChange': 1.1, 'netChange': 139.5, 'bid': 12820.6, 'offer': 12821.6, 'updateTime': '17:29:42', 'delayTime': 0, 'streamingPricesAvailable': True, 'marketStatus': 'TRADEABLE', 'scalingFactor': 1}}, {'position': {'contractSize': 1.0, 'createdDate': '2022/05/26 20:19:08:000', 'dealId': 'DIAAAAJHJD2M8AG', 'dealSize': 1.0, 'direction': 'SELL', 'limitLevel': None, 'openLevel': 12250.1, 'currency': 'GBP', 'controlledRisk': False,

# connecting to the db (azure table storage)

In [1]:
from azure.data.tables import TableServiceClient
from azure.core.exceptions import ResourceNotFoundError, ResourceExistsError
from config import SA_KEY, SA_NAME

# connect to table storage
connection_string = f"DefaultEndpointsProtocol=https;AccountName={SA_NAME};AccountKey={SA_KEY};EndpointSuffix=core.windows.net"
service = TableServiceClient.from_connection_string(conn_str=connection_string)


In [2]:
table = service.get_table_client('translation')
# table.get_entity(partition_key="trade", row_key=str({"hello": "anna"}))

In [12]:
# while True:
translations_list = []

next_pk = None
next_rk = None
i=0
while True:
    entities=table.query_entities("PartitionKey eq 'ig'", next_partition_key = next_pk, next_row_key = next_rk)
    i+=1
    for entity in entities:
        translations_list.append(
            {
                "epic": entity["RowKey"],
                "instrument_name": entity["instrument_name"],
                "trading_view_name": None
                }
        )

    if hasattr(entities, 'x_ms_continuation'):
        x_ms_continuation = getattr(entities, 'x_ms_continuation')
        next_pk = x_ms_continuation['nextpartitionkey']
        next_rk = x_ms_continuation['nextrowkey']
    else:
        break;

In [14]:
import json

In [15]:
with open ("translations_list.json", "w") as f:
    f.write(json.dumps(translations_list))
    

In [ ]:
table.crea

In [5]:
table.query_entities("PartitionKey eq 'ig'")

<iterator object azure.core.paging.ItemPaged at 0x7ff298cf6be0>

In [ ]:
table.create_entity({
            "PartitionKey": "trade",
            "RowKey": "hello",
            "alert": "hello"
        })

In [ ]:
table = service.get_table_client('translation')

epic_name = "TEST.EPIC.IDX"
instrument_name = "Test Instrument Name/USD"

entity = {
    "PartitionKey": "ig",
    "RowKey": epic_name,
    "instrument_name": instrument_name,
    }

# create a new entity in the table
try:
    table.create_entity(entity)
except ResourceExistsError as e:
    print(e)

: 

In [38]:
# retrieve the entity
try:
    print(table.get_entity("ig", epic_name))
    
except ResourceNotFoundError as e:
    print(e)

{'PartitionKey': 'ig', 'RowKey': 'TEST.EPIC.IDX', 'instrument_name': 'Test Instrument Name/USD'}


In [39]:
# delete the entity
table.delete_entity("epic_name", epic_name ) # doesn't error on not found

# DailyBan Logic

In [40]:
from config import DEAL_URL
import datetime as dt
import requests
from utils.utils import get_auth_headers, get_account, get_open_positions, close_positions, send_email
from azure.data.tables import TableServiceClient
from azure.core.exceptions import ResourceNotFoundError, ResourceExistsError
from config import SA_KEY, SA_NAME

# connect to table storage
connection_string = f"DefaultEndpointsProtocol=https;AccountName={SA_NAME};AccountKey={SA_KEY};EndpointSuffix=core.windows.net"
service = TableServiceClient.from_connection_string(conn_str=connection_string)


In [56]:
# Get transactions for today

today = dt.date.today().isoformat()

url = f"{DEAL_URL}/history/transactions?type=ALL_DEAL&from={today}&pageSize=200"
print(url)
headers = get_auth_headers()
headers["version"] = "2"
resp = requests.get(url, headers = headers)
resp.json()




https://api.ig.com/gateway/deal/history/transactions?type=ALL_DEAL&from=2022-07-05&pageSize=200


{'transactions': [{'date': '2022-07-05',
   'dateUtc': '2022-07-05T15:14:04',
   'openDateUtc': '2022-07-05T15:13:17',
   'instrumentName': 'Spot Gold',
   'period': 'DFB',
   'profitAndLoss': '£-0.47',
   'transactionType': 'DEAL',
   'reference': 'DIAAAAJYZTJ77BD',
   'openLevel': '1768.06',
   'closeLevel': '1767.59',
   'size': '+1',
   'currency': '£',
   'cashTransaction': False},
  {'date': '2022-07-05',
   'dateUtc': '2022-07-05T15:14:02',
   'openDateUtc': '2022-07-05T15:11:30',
   'instrumentName': 'Oil - US Crude',
   'period': 'DFB',
   'profitAndLoss': '£-0.55',
   'transactionType': 'DEAL',
   'reference': 'DIAAAAJYZSLP7A9',
   'openLevel': '9878',
   'closeLevel': '9875.8',
   'size': '+0.25',
   'currency': '£',
   'cashTransaction': False},
  {'date': '2022-07-05',
   'dateUtc': '2022-07-05T15:10:21',
   'openDateUtc': '2022-07-05T14:58:08',
   'instrumentName': 'Oil - US Crude',
   'period': 'DFB',
   'profitAndLoss': '£-49.65',
   'transactionType': 'DEAL',
   'refer

In [57]:
balance = get_account()["balance"]
summed = {}

for i in resp.json()['transactions']:
    instrument_name = i["instrumentName"]
    pl = float(i["profitAndLoss"][1:])
    if summed.get(instrument_name):
        summed[instrument_name] = summed[instrument_name] + pl
    else:
        summed[instrument_name] = pl


In [58]:
summed

{'Spot Gold': -235.64,
 'Oil - US Crude': -172.25,
 'Natural Gas': -36.0,
 'British American Tobacco PLC (LSE)': 121.80000000000001,
 'Greggs PLC': 106.25,
 'Corn': -99.5,
 'BAE Systems PLC': 285.0,
 'Sainsbury (J) PLC': 6.4,
 'Germany 40': -8.45}

In [59]:
# [i for i in map(lambda x: x, summed)]
close = []
for k,v in summed.items():
    pl_ratio = v/balance
    if pl_ratio <= -0.02:
        close.append({'instrument_name': k, 'pl_ratio': pl_ratio, 'balance': v})
close

[{'instrument_name': 'Spot Gold',
  'pl_ratio': -0.02856838031686606,
  'balance': -235.64},
 {'instrument_name': 'Oil - US Crude',
  'pl_ratio': -0.020883141697420553,
  'balance': -172.25}]

In [ ]:
table = service.get_table_client('trades')

In [60]:
table = service.get_table_client('translation')

for i in close:

    result = [i for i in table.query_entities(query_filter=f"instrument_name eq '{i['instrument_name']}'")]
    if len(result) == 1:
        i["epic"] = result[0]["RowKey"]
    elif len(result) == 0:
        send_email(f"No epic found for instrument {i['instrument_name']}", "Missing epic / instrument name translation.")
        # exit from func
    
close
    

[{'instrument_name': 'Spot Gold',
  'pl_ratio': -0.02856838031686606,
  'balance': -235.64,
  'epic': 'CS.D.USCGC.TODAY.IP'},
 {'instrument_name': 'Oil - US Crude',
  'pl_ratio': -0.020883141697420553,
  'balance': -172.25,
  'epic': 'CC.D.CL.USS.IP'}]

In [61]:
conditions = [(['market', 'epic'], i['epic']) for i in close]

In [62]:
conditions

[(['market', 'epic'], 'CS.D.USCGC.TODAY.IP'),
 (['market', 'epic'], 'CC.D.CL.USS.IP')]

In [64]:
for c in conditions:
    print(c[1])
    positions = get_open_positions(c)
    print(len(positions))
    print(positions)
    # msg = close_positions(positions)
    # print(msg)


CS.D.USCGC.TODAY.IP
0
[]
CC.D.CL.USS.IP
0
[]
